In [ ]:
import csv 
import json 

def csv_to_json(csvFilePath, jsonFilePath):
    jsonArray = []
      
    #read csv file
    with open(csvFilePath, encoding='utf-8') as csvf: 
        #load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf) 

        #convert each csv row into python dict
        for row in csvReader: 
            #add this python dict to json array
            jsonArray.append(row)
  
    #convert python jsonArray to JSON String and write to file
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
        jsonString = json.dumps(jsonArray, indent=4)
        jsonf.write(jsonString)
          
csvFilePath = r'data.csv'
jsonFilePath = r'data.json'
csv_to_json('data/databern.csv', 'data/bern.json')


In [ ]:
#import piplite
#await piplite.install('ipywidgets')
#await piplite.install('pandas')
#await piplite.install('matplotlib')
#await piplite.install('geopy')
#await piplite.install('pivottablejs')
#await piplite.install('ipyleaflet')
#await piplite.install('gender_guesser')
#!pip install pandas
import piplite
await piplite.install('pandas')
await piplite.install('geocoder')
await piplite.install('ipyleaflet')
await piplite.install('ipywidgets')

import pandas as pd
from utils.prepare_data import getJSON


In [ ]:
data = getJSON('data/bern.json')

df = pd.json_normalize(data[200:220])
pd.DataFrame(df)

In [ ]:
import geocoder

def getGeolocalisationPlace(place):
    """
    Give the geolocation of a giving place.
    The geolocalisation is search in differents local files :
    - location register of edition humboldt
    - location register from GeoName
    If the place isn't already stored in one of this file, then
    a request is sent to the GeoName API.
    :param place: str
    :return: coverage_location
    :rtype: dict
    """
    ortsregister = getJSON('data/edh_ortsregister.json')
    or_geoname = getJSON('data/geoname_ortsregister.json')
    
    coverage_location = {}
    from_geoname = {}
    try :
        for o in ortsregister:
            # If the place of the ortsregister of the edh is the same of the giving place
            # then add the diverse propreties of ortsregister's place to the dict
            # coverage_location
            if place == o['properties']['ContentHeader'] :
                coverage_location["key"] = o['properties']['key']
                coverage_location['geoname_id'] = o['properties']['geoname_id']
                coverage_location['address'] = o['properties']['ContentHeader']
                coverage_location['coordinates'] = o['geometry']['coordinates']
    
        # If the is no place in the ortsregister equal of the giving place
        # then the coverage_location dict stayed empty and we check if the place
        # already was stored in the geoname_ortsregister
        if bool(coverage_location)== False :
            for o in or_geoname:
                for i in o:
                    if place == o[i]["address"]:
                        try:
                            coverage_location["key"] = o[i]['key']
                        except: pass
                        coverage_location['geoname_id'] = o[i]['geoname_id']
                        coverage_location['address'] = i
                        coverage_location['coordinates'] = o[i]['coordinates']

        # If the giving place is neither in the ortsregister of the edh and in
        # stored places from geoname file, then we send a request to geoname to
        # get a geolocation      
        if bool(coverage_location)== False :
            location = geocoder.geonames(place, key='dumont', featureClass='P')
            coverage_location['geoname_id'] = str(location.geonames_id)
            coverage_location['address'] = location.address
            coverage_location['coordinates'] = [location.lng, location.lat]
        
            for ort in ortsregister:
                if ort['properties']['geoname_id'] == str(coverage_location['geoname_id']):
                    coverage_location['key'] = ort['properties']['key']
            from_geoname[place] = coverage_location
        
        # If some geolocalisation from geoname weren't already 
        # stored in some local file, then they are added in the
        # proper geoname_ortsregister.json
        if bool(from_geoname) == True:
            d = getJSON("data/geoname_ortsregister.json")
            d.append(from_geoname)            
            writeJSON("data/geoname_ortsregister.json", d)
    except :
        print(place)
    
    return coverage_location

In [ ]:
#getGeolocalisationPlace("Brussel")
or_geoname = getJSON('data/geoname_ortsregister.json')
coverage_location = {}
place = 'Percy-en-Auge'
test= {"Freiburg": {"geoname_id": "2925177", "address": "Freiburg im Breisgau", "coordinates": ["7.85222", "47.9959"], "key": "H0004809"}}, {"Leamington": {"geoname_id": "2644737", "address": "Royal Leamington Spa", "coordinates": ["-1.52", "52.2852"]}}, {"Plymouth": {"geoname_id": "2186239", "address": "New Plymouth", "coordinates": ["174.08333", "-39.06667"]}}, {"Vlissingen": {"geoname_id": "2745392", "address": "Flushing", "coordinates": ["3.57361", "51.4425"]}}, {"Percy-en-Auge": {"geoname_id": "2988016", "address": "Percy-en-Auge", "coordinates": ["-0.06354", "49.05312"]}}, {"Brussel": {"geoname_id": "2800866", "address": "Brussels", "coordinates": ["4.34878", "50.85045"], "key": "H0005371"}}

for o in test:
                for i in o:
                    if place == o[i]["address"]:
                        try:
                            coverage_location["key"] = o[i]['key']
                        except: pass
                        coverage_location['geoname_id'] = o[i]['geoname_id']
                        coverage_location['address'] = i
                        coverage_location['coordinates'] = o[i]['coordinates']

print(coverage_location)

In [ ]:
import json

def writeJSON(file, data):
    """
    Store JSON data in a JSON file 
    :param file: str
    :param data: dict
    """
    with open(file, mode='w') as f:
        json.dump(data, f)

dataWithGeo= []
index = 0
data = getJSON('data/bern.json')

for i in data:
    item = {}
    item['link'] = data[index]['ï»¿Link'].split('(')[1].split(')')[0]
    item['title'] = data[index]['ï»¿Link'].split('[')[1].split(']')[0] # ï»¿Link

    if ',' in data[index]['Erscheinungsort']:
        place = data[index]['Erscheinungsort'].split(',')[0]
    elif ';' in data[index]['Erscheinungsort']:
        place = data[index]['Erscheinungsort'].split(';')[0]
    else : place = data[index]['Erscheinungsort']
    item['pubplace'] = getGeolocalisationPlace(place)
    item['year'] = data [index]['Jahr']
    item['language'] = data[index]['Sprache']
    index += 1
    dataWithGeo.append(item)

writeJSON('data/bern_withgeo.json', dataWithGeo)


# Data visualisations

In [ ]:
from ipyleaflet import Map, Marker, Popup, CircleMarker
from ipywidgets import HTML, Output, HBox, Layout

def allOnAMap(data):
    cities = {}
    marker = None
    coordinates = []
    m= Map(
            zoom=1.5,
            layout=Layout(width='80%', height='500px'),
            close_popup_on_click=False
            )
    
    for i in data:
            try :
                if i["pubplace"]["address"] not in cities:
                    city = i["pubplace"]["address"]
                    cities[city] = {}
                    cities[city]["message"] = "<b>"+ i["year"] + " </b> " + i["title"] + "<br><i>"+"</i> <br> <a href=\""+ i["link"] + "\" target=\"_blank\">online</a> <hr>"
                    cities[city]["coordinates"] = [i['pubplace']['coordinates'][1], i['pubplace']['coordinates'][0]] 
                elif i["pubplace"]["address"] in cities:
                    city = i["pubplace"]["address"]
                    cities[city]["message"] = cities[city]["message"] + "<b>"+ i["year"] + " </b> " + i["title"] + "<br><i>"+"</i> <br> <a href=\""+ i["link"] + "\" target=\"_blank\">online</a> <hr>"
            except : pass
        # Coordinates to create a dynamic map boundaries
    try:
        for i in cities.keys():
            if type(cities[i]["coordinates"][0]) == float and type(cities[i]["coordinates"][1]) == float:
                coordinates.append([float(cities[i]["coordinates"][0]), float(cities[i]["coordinates"][1])])
            elif type(cities[i]["coordinates"][0]) == str and type(cities[i]["coordinates"][1]) == str:
                coordinates.append([float(cities[i]["coordinates"][0]), float(cities[i]["coordinates"][1])])
    
        coordinates = numpy.array(coordinates)
        data_frame = pd.DataFrame(coordinates, columns=['Lat', 'Long'])
        sw = data_frame[['Lat', 'Long']].min().values.tolist()
        ne = data_frame[['Lat', 'Long']].max().values.tolist()
        m.fit_bounds([sw, ne])
    except: pass


    # Mapmarker and popup message
    for i in cities.keys():
            try :
                # Create the message of the popup
                message = HTML()
                if cities[i]["message"].count("<hr>") <5 :
                    message.value = cities[i]["message"]
                else : 
                    message.value = str(cities[i]["message"].count("<hr>")) + " publications. There are too many results to show them all here."
                message.description = i.upper()

                # Create the marker
                marker = CircleMarker(location=(cities[i]["coordinates"][0], cities[i]["coordinates"][1]))
                radius = cities[i]["message"].count("<hr>")+3
                if radius > 10:
                    radius = 12
                marker.radius = radius
                marker.fill_opacity = 0.8
                marker.fill_color = '#3E81B8'
                marker.stroke = False

                # Add marker on the map
                m.add_layer(marker)
                marker.popup = message
            except: pass
    display(m)
    
withgeo = getJSON('data/bern_withgeo.json')
allOnAMap(withgeo)